In [9]:
import time
import pandas as pd
import numpy as np
from datetime import datetime
import re
import locale
#import pyautogui

from utils import *

import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains

Define: ```Elemento a buscar``` ```Estado``` ```distancia``` ```n_scrolls_cada_vez```
 ```Lista a incluir``` 
 ```Lista a excluir```

In [10]:
#ejecutar con el entorno .env
elemento_a_buscar = 'iphone 15'
n_iphone = elemento_a_buscar.split()[1] # para otros elementos modificlar el bucle que verifica si el item es correcto

#funcion para seleccionar los articulos que quiero:
# si una la tiene o no la tiene se incluye o excluye.
# por ejemplo si buscadas = 15 y 16
#                excluyentes = funda cristal
#                'funda iphone 15' se excluye
#                'iphone'      no se incluye porque no tiene el 15 ni el 16
cadenas_buscadas = (
                    n_iphone,
                    n_iphone + ',',
                    n_iphone + '.'
                    )  # la coma es para que python lo vea como una tupla
cadenas_excluyentes = (
                    "funda",
                    "fundas",
                    "carcasa",
                    "carcasas",
                    "protector",
                    "cristal",
                    "3", "4", "5", "6",
                    "4s", "5s", "6s"
                    "16gb",
                    "caja",
                    "macbook"
)

# 'new', 'as_good_as_new', 'good'
#Solo uno cada vez!
estado = 'new'
# Para buscar se podrían poner juntos sin espacios con comas. 'new,as_good_as_new,good'
# Pero para guardarlos en la tabla no se puede guardar el estado extraido. 
distancia = '60' # distancia en km
precio_min = 250

carpeta ="1_datos_raw"
n_excluidos_seguidos_max = 30
n_scrolls_cada_vez = 15 
if type(cadenas_buscadas) is not tuple:
    print("cadenas_buscadas no es tupla")
if type(estado) is not str:
    print("estado no es str") 

In [11]:
#Abre la venana
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

ele_1 = elemento_a_buscar.split()[0]
ele_2 = elemento_a_buscar.split()[1]
#driver.get('https://es.wallapop.com');
# para Madrid no cambiar las coordenadas
url = 'https://es.wallapop.com/app/search?filters_source=quick_filters&keywords=' + ele_1 + '%20' + ele_2 + '&longitude=-3.69196&latitude=40.41956&distance='+ str(distancia) + '000&condition=' + str(estado)
driver.get(url)
print(url)
time.sleep(1)
driver.maximize_window()  
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(1)

https://es.wallapop.com/app/search?filters_source=quick_filters&keywords=iphone%2015&longitude=-3.69196&latitude=40.41956&distance=60000&condition=new


In [12]:
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
    ).click()


In [13]:
time.sleep(8)
# clico 3 veces fuera del cuadrado para saltar el tutorial
x_coord = 1600  # Coordenada X deseada
y_coord = 600  # Coordenada Y deseada
pyautogui.click(x=x_coord, y=y_coord)
time.sleep(2)
x_coord = 1600  # Coordenada X deseada
y_coord = 600  # Coordenada Y deseada
pyautogui.click(x=x_coord, y=y_coord)
time.sleep(2)
x_coord = 1600  # Coordenada X deseada
y_coord = 600  # Coordenada Y deseada
pyautogui.click(x=x_coord, y=y_coord)
time.sleep(4)

In [14]:
# no se que hace. creo que cargar 40 articulos más
# button = WebDriverWait(driver, 6).until(
#     EC.element_to_be_clickable((By.XPATH, "//walla-button[contains(@class, 'w-100')]"))
# )
# # Haz clic en el botón
# button.click()
# time.sleep(2)

In [15]:
# Clicamos para ver más
boton_ver_mas = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "btn-load-more")))
boton_ver_mas.click()
time.sleep(2)

# button = WebDriverWait(driver, 6).until(
#     EC.element_to_be_clickable((By.XPATH, "//walla-button[contains(@class, 'w-100')]"))

In [16]:
# Creación de la tabla con las columnas que buscamos
try:
    del df
except NameError:
    pass
columnas = ['id', 'nombre', 'precio', 'estado', 'reservado', 'url']
df = pd.DataFrame(columns=columnas)

In [17]:
time.sleep(2)
x_coord = 1600
y_coord = 780 
pyautogui.click(x=x_coord, y=y_coord)

# Scrolling

In [18]:
# Clicamos para ver más y hacemos scroll down n veces
n = 0
while n < n_scrolls_cada_vez:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    n = n + 1

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.86)
Stacktrace:
	GetHandleVerifier [0x00AF33E3+25059]
	(No symbol) [0x00A7CDE4]
	(No symbol) [0x0095BEC3]
	(No symbol) [0x0093D93B]
	(No symbol) [0x009C800F]
	(No symbol) [0x009DAE49]
	(No symbol) [0x009C1C96]
	(No symbol) [0x00993FAC]
	(No symbol) [0x00994F3D]
	GetHandleVerifier [0x00DE5543+3113795]
	GetHandleVerifier [0x00DFA20A+3198986]
	GetHandleVerifier [0x00DF29E2+3168226]
	GetHandleVerifier [0x00B93250+680016]
	(No symbol) [0x00A8572D]
	(No symbol) [0x00A829D8]
	(No symbol) [0x00A82B75]
	(No symbol) [0x00A757D0]
	BaseThreadInitThunk [0x762D7BA9+25]
	RtlInitializeExceptionChain [0x77A3C0CB+107]
	RtlClearBits [0x77A3C04F+191]


In [ ]:
def item_reservado(elem):
    # Asumiendo que la clase 'reservado' en algún elemento interno indica que el artículo está reservado
    try:
        elemento_reservado = elem.find_element(By.CSS_SELECTOR, ".clase_indicativa_de_reservado")
        return True
    except:
        return False

In [ ]:
def es_precio_ok(precio: str, precio_min: int):
    precio = precio.replace(".", "TEMP").replace(",", ".").replace("TEMP", ",")
    precio = float(precio)
    if precio >= precio_min: 
        return 1
    else:
        return 0

In [ ]:
# Extraigo los datos y los añado en la tabla. 
elementos = driver.find_elements(By.CSS_SELECTOR, "a.ItemCardList__item")
total_elementos =  len(elementos)
n_excluidos_seguidos = 0
for index, elem in enumerate(elementos):
    if n_excluidos_seguidos >= n_excluidos_seguidos_max:
        break
    pos = (str(index) + "/" + str(total_elementos))
    url_articulo = elem.get_attribute('href')
    id_articulo = url_articulo.split('-')[-1]
    if not ya_existe_articulo(id_articulo, df):
        nombre = elem.get_attribute('title')
        nombre = nombre.lower()
        if is_iphone_xx(nombre, n_iphone, cadenas_buscadas, cadenas_excluyentes):
            precio = elem.find_element(By.CSS_SELECTOR, ".ItemCard__price").text.strip()
            precio = precio.split(' ')[0]
            if es_precio_ok(precio, precio_min):
                reservado = item_reservado(elem)  
                # Guarda en la tabla
                print(pos, "++ guardado   ++ " + nombre)
                df.loc[index] = [id_articulo, nombre, precio, estado, reservado, url_articulo]
                n_excluidos_seguidos = 0
            else:
                print(pos, "--precio bajo -- ", precio, nombre)
        else:
            n_excluidos_seguidos = n_excluidos_seguidos + 1
            print(pos, "## excluido   ## ", nombre)
    else:
        print(pos, "* ya existe:  ** ", nombre, id_articulo)
        n_excluidos_seguidos = 0
        
    
df = df.reset_index(drop=True)
print(df)

0/613 ++ guardado   ++ iphone 15 128gb negro
1/613 ++ guardado   ++ iphone 15 pro 128gb natural
2/613 ++ guardado   ++ iphone 15 pro 128gb natural
3/613 ++ guardado   ++ iphone 15 pro 128gb titanio azul
4/613 ++ guardado   ++ iphone 15 pro
5/613 ++ guardado   ++ iphone 15 pro max 256gb negro
6/613 ++ guardado   ++ iphone 15 pro 256g
7/613 ++ guardado   ++ apple iphone 15 128gb verde - revisado
8/613 ++ guardado   ++ apple iphone 15 512gb azul - revisado
9/613 ++ guardado   ++ apple iphone 15 256gb azul - revisado
10/613 ++ guardado   ++ apple iphone 15 128gb azul - revisado
11/613 ++ guardado   ++ apple iphone 15 256gb negro - revisado
12/613 ++ guardado   ++ apple iphone 15 128gb negro - revisado
13/613 ++ guardado   ++ iphone 15 pro max 512 gb
14/613 ++ guardado   ++ iphone 15 pro max 256gb
15/613 ## excluido   ##  fundas iphone 15
16/613 ++ guardado   ++ apple iphone 15 256gb bat 100% garantia
17/613 ## excluido   ##  iphone 14 plus
18/613 ## excluido   ##  iphone 14 pro 
19/613 ++ 

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
df

,id,nombre,precio,estado,reservado,url
0,1073757376,iphone 15 128gb negro,"599,00",good,False,https://es.wallapop.com/item/iphone-15-128gb-negro-1073757376
1,1073765761,iphone 15 pro 128gb natural,"799,00",good,False,https://es.wallapop.com/item/iphone-15-pro-128gb-natural-1073765761
2,1073756019,iphone 15 pro 128gb natural,"799,00",good,False,https://es.wallapop.com/item/iphone-15-pro-128gb-natural-1073756019
3,1073762896,iphone 15 pro 128gb titanio azul,"849,00",good,False,https://es.wallapop.com/item/iphone-15-pro-128gb-titanio-azul-1073762896
4,1073593723,iphone 15 pro,"950,00",good,False,https://es.wallapop.com/item/iphone-15-pro-1073593723
...,...,...,...,...,...,...
170,1046518386,iphone 15 plus 128 gb gris espacial + caja,"753,95",good,False,https://es.wallapop.com/item/iphone-15-plus-128-gb-gris-espacial---caja-1046518386
171,1048257356,iphone 15 128gb azul + cable + caja 100% bat,"623,95",good,False,https://es.wallapop.com/item/iphone-15-128gb-azul---cable---caja-100-bat-1048257356
172,1035824899,iphone 15 pro max 1 tb,"1000,00",good,False,https://es.wallapop.com/item/iphone-15-pro-plus-1-tb-1035824899
173,1046524754,movil iphone 15 plus negro 256gb salud bateria 100% + caja + cable,"853,95",good,False,https://es.wallapop.com/item/movil-iphone-15-plus-negro-256gb-salud-bateria-100---caja---cable-1046524754


In [ ]:
hoy_formateada = datetime.now().strftime("%Y%m%d")
print(hoy_formateada)
print(elemento_a_buscar)
print(estado)
nombre_archivo_pkl = hoy_formateada + '_' + elemento_a_buscar + '_' + estado
df.to_csv(carpeta + '/' + nombre_archivo_pkl + '.csv')

20241127
iphone 15
good


In [ ]:
try:
    df = pd.read_csv(carpeta + '/' + nombre_archivo_pkl + '.csv')
    print("csv guardado")
    driver.quit()
except:
    print("Error leyendo archivo! puede que no se haya guardado bien")


csv guardado
